In [ ]:
import os
import torch
import numpy as np
import sys

sys.path.append(os.path.join(os.path.dirname("__file__"), "../"))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading

In [ ]:
iterations = 5
ori_data = np.load(
    "../OUTPUT/stocks-sample-timewak-24/samples/stock_norm_truth_24_test.npy"  # Replace this
)
fake_data = np.load(
    "../OUTPUT/stocks-sample-timewak-24/ddpm_fake_norm_stocks-sample-timewak-24.npy"  # Replace this
)

## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [ ]:
context_fid_score = []

for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[: ori_data.shape[0]])
    context_fid_score.append(context_fid)
    print(f"Iter {i}: Context-FID = {context_fid}")

mean, sigma = display_scores(context_fid_score)
print(f"Final Score: {mean:.3f} \xb1 {sigma:.3f}")

## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [ ]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [ ]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name="CrossCorrelLoss")
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f"Iter {i}: Correlational = {loss.item()}")

mean, sigma = display_scores(correlational_score)
print(f"Final Score: {mean:.3f} \xb1 {sigma:.3f}")